In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import random
from ipyleaflet import Map, Circle
import seaborn as sns
sns.set()
from IPython.display import display

# # Generate more data
# for index, row in df.iterrows():
#     for i in range(4):
#         df.loc[len(df)] = [
#             row['lat'] + random.uniform(-2/100000, 2/100000),
#             row['lng'] + random.uniform(-2/100000, 2/100000),
#             row['heading'] + random.uniform(-1, 1),
#             row['isMatch']
#         ]
# norm_df = (df - df.min()) / (df.max() - df.min())

# Prepare the Data

In [ ]:
df = pd.read_csv('places-small.csv')
#shuffel the places
df = df.sample(frac=1).reset_index(drop=True)

inputMatrix = df[['lat', 'lng', 'heading']].as_matrix()
target = df['isMatch'].as_matrix()

df

# Visualize the Data

In [ ]:
df.hist()
plt.show()

map = Map(center=[df.lat.mean(), df.lng.mean()], zoom=17)

def addPoint(map, lat, lng, isMatch, predicted_match=None):
    if not (predicted_match is None) and predicted_match != isMatch:
        color = '#f44'
    else:
        color = '#44F' if isMatch else '#999'
    map.add_layer(Circle(location=[lat, lng], radius=8, color=color, opacity=.7, fill_opacity=.7, fill_color=color))

for index, row in df.iterrows():
    addPoint(map, lat=row.lat, lng=row.lng, isMatch=row.isMatch)

map

# Train the Model
### Sigmoid Activation Function
![python-trends](img/sigmoid.png)

In [ ]:
def get_model():
    model = Sequential()
    model.add(Dense(30, input_dim=len(inputMatrix[0]), activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
model = get_model()

training_data = model.fit(inputMatrix, target, validation_split=0.5, epochs=300, batch_size=10, verbose=False)

fig, axis = plt.subplots(1,2)
fig.set_size_inches(14, 4.5)
axis[0].plot(training_data.history['val_loss'], 'g', training_data.history['loss'], 'r')
axis[0].set_xlabel('Epochs')
axis[0].set_ylabel('Loss')
axis[1].plot(training_data.history['val_acc'], 'g', training_data.history['acc'], 'r')
axis[1].set_xlabel('Epochs')
axis[1].set_ylabel('Accuracy')
plt.show()

print('Accuracy: %.2f%%' % (training_data.history['val_acc'][-1]*100))

# Visualize the Model

In [ ]:
map = Map(center=[df.lat.mean(), df.lng.mean()], zoom=17)

for index, input_row in enumerate(inputMatrix):
    predicted_match = model.predict(np.array([input_row])) > 0.5;
    raw_row = df.loc[index]
    addPoint(map, lat=raw_row.lat, lng=raw_row.lng, isMatch=raw_row.isMatch, predicted_match=predicted_match)

map